## 13.2.0 前言

在上一节的最后，我们提到图13.5：**。如果潜在变量是离散的，那么我们得到了隐马尔科夫模型（hidden Markov model）**

一个简单的例子：
该例子来自[维基百科](https://en.wikipedia.org/wiki/Hidden_Markov_model)。我进行了一些改变。

王小花和李二狗是异地恋。每天王小花都会通过电话了解李二狗做了什么。李二狗只做四个活动：散步、购物、打游戏、清理房间。他选择做什么只凭当天的天气。王小花对李二狗所住的地方的天气不了解，但知道总体的趋势。而天气情况有三种：晴天、多云、雨天。这些天气是王小花不知道的，李二狗会根据天气情况有一定的概率进行下面的活动：“散步”、“购物”、“打游戏”、“清理房间”。这些李二狗会通过电话给王小花汇报，所以这些数据王小花是知道的。这整个系统就是一个**隐马尔可夫模型HMM**

王小花知道这个地区的总的天气趋势,并且平时知道李二狗会做的事情.也就是说这个隐马尔可夫模型的参数是已知的.可以用程序语言(Python)写下来:
```python
"""
数据结构采用集合和字典
"""

# 隐藏状态数目，三个状态：晴天、多云、雨天
states = ('Sunny', 'Cloudy', 'Rainy')

# 每个隐藏状态下可能的观察值
observations = ('walk', 'shop', 'game', 'clean')

# 初始状态空间的概率分布
start_probability = {'Sunny': 0.3, 'Cloudy': 0.1, 'Rainy': 0.6}

# 与时间无关的状态转移概率矩阵
transition_probability = {
    'Sunny': {'Sunny': 0.5, 'Cloudy': 0.3, 'Rainy': 0.2},
    'Cloudy': {'Sunny': 0.3, 'Cloudy': 0.5, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.2, 'Cloudy': 0.1, 'Rainy': 0.7},
}

# 给定状态下，观察值概率分布,发射概率
emission_probability = {
    'Sunny': {'walk': 0.4, 'shop': 0.3, 'game': 0.2, 'clean': 0.1},
    'Cloudy': {'walk': 0.3, 'shop': 0.4, 'game': 0.1, 'clean': 0.2},
    'Rainy': {'walk': 0.1, 'shop': 0.2, 'game': 0.4, 'clean': 0.3}, 
}
```

- start_probability代表了王小花对于李二狗第一次给她打电话时的天气情况的不确定性(王小花知道的只是那个地方平均起来下雨多些)。
- transition_probability表示了状态转移的概率。例如今天是晴天那么明天晴天的概率是0.5，多云是0.3，下雨是0.2。
- emission_probability表示了李二狗每天做某事的概率【发射概率】（假设他一天只做这一件）。例如晴天，那么他散步的概率是0.4，购物时0.3，以此类推。

王小花和李二狗通了三天电话。第一天李二狗去散步了，第二天他去购物了，第三天他清理房间了。那么问题来了

- （1）“散步、购物、清理”的总的概率是多少？(注：这个问题对应于HMM的基本问题之一：已知HMM模型$\lambda$及观察序列$O$，如何计算$P(O|\lambda)$？)
- （2）这个观察序列的隐藏状态序列（晴天/多云/雨天）又是什么？（注：这个问题对应HMM基本问题之二：给定观察序列$O=O_1,O_2,…O_T$以及模型$\lambda$,如何选择一个对应的隐藏状态序列$S=q_1,q_2,…q_T$，使得$S$能够最为合理的解释观察序列$O$？）
-  （3）如何调整模型参数, 使得$P(O|λ)$最大？这个问题事实上就是给出很多个观察序列值，来训练以上几个参数的问题。

![](https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH13/HMM1.png)

**下图表示了隐藏转态序列之间的转移**
![](https://gitee.com/data2world/PRML/raw/master/CH13/13-6.png)

有时⽐较有⽤的做法是将图13.6所⽰的状态转移图在时间上展开。这给出了潜在变量之间转移的另⼀种表⽰⽅法，被称为晶格图（lattice diagram）或者格⼦图（trellis diagram）。图13.7给出了隐马尔科夫模型的晶格图。

![](https://gitee.com/data2world/PRML/raw/master/CH13/13-7.png)



### 在开始下面一系列的推到之前，我们先总结一下HMM的基本组成要素：

**一个HMM模型可以用5个元素来描述，包过2个状态集合和3个概率矩阵。其分别为：**

- **可观测状态序列：** 如散步、购物、清理等。用数学符号表示：$X=\{x_1,..., x_N\}$
- **隐藏状态序列：**如王小花不知道的天气情况。同样的用数学符号表示为：$Z=\{z_1,...,z_N\}$
- **初始状态概率矩阵：** 上面例子的start_probability变量。用符号$\pi$表示。
- **隐藏状态概率转移矩阵：**上面例子的transition_probability变量。用符号$A$表示。
- **观测状态转移概率矩阵：**上面例子的emission_probability变量。用符号$\phi$表示。

从⽽观测变量和潜在变量上的联合概率分布为:

$$
p(X,Z|\theta) = p(z_1|\pi)\left[\prod\limits_{n=2}^N p(z_n|z_{n−1},A)\right]\prod\limits_{m=1}^Np(x_m|z_m,\phi) \tag{13.10}
$$

**其中$\theta=\{\pi,A,\phi\}$**

## 13.2.1 用于HMM的最大似然法

这一小节我们使用最大似然法求解HMM的参数。由式（13.10）对潜变量求和得：




























## 13.2.2 前向后向算法

## 13.2.3 用于HMM的加和-乘积算法

## 13.2.4 缩放因子

## 13.2.5 维比特算法

## 13.2.6 隐马尔科夫模型扩展

## 参考